In [1]:
import requests
import os
import json
import pandas as pd
import numpy as np
import getpass
import neo4j
from neo4j import GraphDatabase
import time

In [2]:
from platform import python_version

print(python_version())

3.9.12


In [3]:
pd.set_option("display.max_rows",2000)

In [4]:
bearer_token = getpass.getpass(prompt = 'BEARER_TOKEN?')

BEARER_TOKEN?········


In [5]:
# Please don't save your twitter bearer token here
# os.environ['BEARER_TOKEN'] = 'xxxxxxx'
# bearer_token = os.environ.get('BEARER_TOKEN')

In [6]:
# print(bearer_token)

In [7]:
def create_url_get_id(usrnm):
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values.
    usernames = usrnm
    user_fields = "user.fields=id,name,username,created_at,public_metrics"
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    return url

In [8]:
def bearer_oauth_get_id(r):
    """
    Method required by bearer token authentication.
    """
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r

In [9]:
def connect_to_endpoint_get_id(url):
    response = requests.request("GET", url, auth=bearer_oauth_get_id,)
#     print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [10]:
def get_id(usrnm):
    url = create_url_get_id(usrnm)
    json_response = connect_to_endpoint_get_id(url)
#     print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response

In [31]:
"""
(@AzukiZen, @worldwide_WEB3), 
less successful projects (@RavePigsNFT), popular people in the space (AB, Ellio, Squid, Vidar), 
and a few normies in the space with low follower count (@Arvo mine lol)
"""
cols = ['id','name','username','created_at','public_metrics.followers_count',\
        'public_metrics.following_count','public_metrics.tweet_count','public_metrics.listed_count']
df_get_id = pd.DataFrame()
# usernames=['VulcanForged','DeRaceNFT','altura_nft','playbigtime','SIPHERxyz','animocabrands',\
#            'zssbecker','elliotrades','AzukiOfficial','GoGalaGames','ultra_io','DefiKingdoms',]
usernames=['VulcanForged','DeRaceNFT','altura_nft','SIPHERxyz','animocabrands',\
           'zssbecker','elliotrades','AzukiOfficial','GoGalaGames','ultra_io','DefiKingdoms',]
# usernames=['altura_nft','SIPHERxyz','ZssBecker','AzukiOfficial','GoGalaGames','ultra_io','DefiKingdoms']
# Make a short list since we are restricted to 15 follower lookup requests per 15 minute period
# usernames=['VulcanForged','DeRaceNFT','playbigtime','SIPHERxyz','animocabrands']
for username in usernames:
#     print(username)
    out_id_data = (get_id('usernames='+username))
#     df_get_id = df_get_id.append(pd.json_normalize(out_id_data['data']),ignore_index=True)
    df_get_id = pd.concat([df_get_id,pd.json_normalize(out_id_data['data'])],ignore_index=True)
df_get_id = df_get_id.loc[:,cols]
df_get_id

,id,name,username,created_at,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count
0,1261684151780081664,Vulcan Forged,VulcanForged,2020-05-16T15:45:41.000Z,109365,915,2658,605
1,1386276828759855109,DeRace,DeRaceNFT,2021-04-25T11:12:19.000Z,180736,454,3153,634
2,1381135276794388482,Altura,altura_nft,2021-04-11T06:41:42.000Z,130388,355,1109,417
3,1402266580357201923,SIPHΞR,SIPHERxyz,2021-06-08T14:10:13.000Z,108022,211,4628,735
4,2478217117,Animoca Brands,animocabrands,2014-05-05T11:15:04.000Z,135310,1266,11772,1334
5,1243470638477697024,Alex Becker 🍊🏆🥇,ZssBecker,2020-03-27T09:31:26.000Z,815978,278,6312,5479
6,948736680554409984,EllioTrades,elliotrades,2018-01-04T02:03:46.000Z,690515,1088,6671,5289
7,1432583226707484676,Azuki,AzukiOfficial,2021-08-31T05:57:24.000Z,259433,393,828,949
8,1288572182444961793,Gala Games,GoGalaGames,2020-07-29T20:29:05.000Z,347427,154,2385,1586
9,992778329685143552,Ultra,ultra_io,2018-05-05T14:49:33.000Z,70913,93,1126,527


In [12]:
def create_url_get_followers(user_id):
    # Replace with user ID below
#     user_id = 2244994945
#     return "https://api.twitter.com/2/users/{}/followers".format(user_id)
    return f"https://api.twitter.com/2/users/{user_id}/followers"

In [13]:
def create_url_get_following(user_id):
    # Replace with user ID below
#     user_id = 2244994945
#     return "https://api.twitter.com/2/users/{}/followers".format(user_id)
    return f"https://api.twitter.com/2/users/{user_id}/following"

In [14]:
# https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a
def get_params_get_followers(next_page,max_results=1000):
    query_params = {
        'user.fields':'id,name,username,created_at',
        'max_results':max_results,
        'pagination_token':next_page
    }
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
#     user_fields = "user.fields:id,name"
    return query_params

In [15]:
# https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a
def get_params_get_following(next_page,max_results=1000):
    query_params = {
        'user.fields':'id,name,username,created_at',
        'max_results':max_results,
        'pagination_token':next_page
    }
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
#     user_fields = "user.fields:id,name"
    return query_params

In [16]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FollowersLookupPython"
    return r

In [17]:
def connect_to_endpoint_get_followers(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
#     print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [18]:
def connect_to_endpoint_get_following(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
#     print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [19]:
def get_followers(id,next_page):
    url = create_url_get_followers(id)
    params = get_params_get_followers(next_page)
    json_response = connect_to_endpoint_get_followers(url, params)
#     print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response

In [20]:
def get_following(id,next_page):
    url = create_url_get_following(id)
    params = get_params_get_following(next_page)
    json_response = connect_to_endpoint_get_followers(url, params)
#     print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response

In [21]:
# get_following(1243470638477697024,None)

In [22]:
# Execute this to get followers (it will take many hours to get them all) - code needs to be modified with sleep functionality

# cols = ['id','name','username','created_at','follow_id','follow_name','follow_username']
# df_followers_all = pd.DataFrame()
# df_followers_meta_all = pd.DataFrame()
# # cycle through the id list and pull followers
# # result limit on api call is 1000 records
# # need to ad
# for _, row in df_get_id.iterrows():

#     df_followers = pd.DataFrame()
#     df_followers_meta = pd.DataFrame()
#     follow_id = row['id']
#     follow_name = row['name']
#     follow_username = row['username']
#     i = 1
#     next_page = None
# # Short list has length of 5. Attempt to get 3 pages of data per id (15 total calls out of 15 limit)
#     while i <= 3:
#         out_id_data = (get_followers(follow_id,next_page))
#         df_followers = df_followers.append(pd.json_normalize(out_id_data['data']),ignore_index=True)
#         df_followers['follow_id'] = follow_id
#         df_followers['follow_name'] = follow_name
#         df_followers['follow_username'] = follow_username
#         df_followers = df_followers.loc[:,cols]
#         df_meta = pd.json_normalize(out_id_data['meta'])
#         df_followers_meta = df_followers_meta.append(df_meta,ignore_index=True)
#         next_page = df_meta['next_token'].values[0]
#         if next_page == None:
#             print('no next_page: ',follow_id,follow_name,follow_username,i)
#             break
#         i+=1

#     df_followers_all = df_followers_all.append(df_followers,ignore_index=True)
#     df_followers_meta_all = df_followers_meta_all.append(df_followers_meta,ignore_index=True)
    
# df_followers_all.reset_index(drop=True,inplace=True)
# df_followers_meta_all.reset_index(drop=True,inplace=True)


In [23]:
# df_meta

In [32]:
cols = ['id','name','username','created_at','following_id','following_name','following_username']
df_following_all = pd.DataFrame()
df_following_meta_all = pd.DataFrame()
# cycle through the id list and pull following
# result limit on api call is 1000 records
# need to ad
overall_loop_cnt = 0
for _, row in df_get_id.iterrows():

    df_following = pd.DataFrame()
    df_following_meta = pd.DataFrame()
    following_id = row['id']
    following_name = row['name']
    following_username = row['username']
    following_count = row['public_metrics.following_count']
    loop_cnt = int(following_count/1000) + 1
    print(following_id,following_name,following_username,following_count,loop_cnt)
    next_page = None
# Limit to 15 calls to the followers twitter endpoint every 15 minutes
    for i in range(loop_cnt):
        overall_loop_cnt +=1
        if overall_loop_cnt > 15 and overall_loop_cnt % 15 == 1:
            print("sleeping for 15 minutes")
            time.sleep(60*15) # sleep for 15 minutes to avoid error: 429 {"title":"Too Many Requests"
        out_id_data = (get_following(following_id,next_page))
        df_following = pd.concat([df_following,pd.json_normalize(out_id_data['data'])],ignore_index=True)
        df_following['following_id'] = following_id
        df_following['following_name'] = following_name
        df_following['following_username'] = following_username
        df_following = df_following.loc[:,cols]
        df_meta = pd.json_normalize(out_id_data['meta'])
        df_following_meta = pd.concat([df_following_meta,df_meta],ignore_index=True)
        try:
            next_page = df_meta['next_token'].values[0]
        except:
            next_page = None

#     df_following_all = df_following_all.append(df_following,ignore_index=True)
    df_following_all = pd.concat([df_following_all,df_following],ignore_index=True)
#     df_following_meta_all = df_following_meta_all.append(df_following_meta,ignore_index=True)
    df_following_meta_all = pd.concat([df_following_meta_all,df_following_meta],ignore_index=True)
    
df_following_all.reset_index(drop=True,inplace=True)
df_following_meta_all.reset_index(drop=True,inplace=True)

1261684151780081664 Vulcan Forged VulcanForged 915 1
1386276828759855109 DeRace DeRaceNFT 454 1
1381135276794388482 Altura altura_nft 355 1
1402266580357201923 SIPHΞR SIPHERxyz 211 1
2478217117 Animoca Brands animocabrands 1266 2
1243470638477697024 Alex Becker 🍊🏆🥇 ZssBecker 278 1
948736680554409984 EllioTrades elliotrades 1088 2
1432583226707484676 Azuki AzukiOfficial 393 1
1288572182444961793 Gala Games GoGalaGames 154 1
992778329685143552 Ultra ultra_io 93 1
1412168764284706822 DeFi Kingdoms DefiKingdoms 101 1


In [25]:
out_id_data

{'data': [{'id': '1451101738438696966',
   'created_at': '2021-10-21T08:23:37.000Z',
   'username': 'PowderHeroes',
   'name': 'Powder'},
  {'id': '823104400373649409',
   'created_at': '2017-01-22T09:45:56.000Z',
   'username': 'shxinjp',
   'name': 'sashei 👺'},
  {'id': '1391838004419964929',
   'created_at': '2021-05-10T19:30:23.000Z',
   'username': 'CryptoOneCafe',
   'name': 'Jess | Harmony Validator 💙'},
  {'id': '984634995582136321',
   'created_at': '2018-04-13T03:30:51.000Z',
   'username': 'CryptoNCoffeee',
   'name': 'Jess💙'},
  {'id': '1380341547154083844',
   'created_at': '2021-04-09T02:08:36.000Z',
   'username': 'traderjoe_xyz',
   'name': 'Trader Joe🔺'},
  {'id': '2555176531',
   'created_at': '2014-06-08T17:06:09.000Z',
   'username': 'CoinbaseSupport',
   'name': 'Coinbase Support'},
  {'id': '2260491445',
   'created_at': '2013-12-24T16:50:12.000Z',
   'username': 'CoinMarketCap',
   'name': 'CoinMarketCap'},
  {'id': '1445334798977830912',
   'created_at': '2021-1

In [26]:
df_following

,id,name,username,created_at,following_id,following_name,following_username
0,1451101738438696966,Powder,PowderHeroes,2021-10-21T08:23:37.000Z,1412168764284706822,DeFi Kingdoms,DefiKingdoms
1,823104400373649409,sashei 👺,shxinjp,2017-01-22T09:45:56.000Z,1412168764284706822,DeFi Kingdoms,DefiKingdoms
2,1391838004419964929,Jess | Harmony Validator 💙,CryptoOneCafe,2021-05-10T19:30:23.000Z,1412168764284706822,DeFi Kingdoms,DefiKingdoms
3,984634995582136321,Jess💙,CryptoNCoffeee,2018-04-13T03:30:51.000Z,1412168764284706822,DeFi Kingdoms,DefiKingdoms
4,1380341547154083844,Trader Joe🔺,traderjoe_xyz,2021-04-09T02:08:36.000Z,1412168764284706822,DeFi Kingdoms,DefiKingdoms
5,2555176531,Coinbase Support,CoinbaseSupport,2014-06-08T17:06:09.000Z,1412168764284706822,DeFi Kingdoms,DefiKingdoms
6,2260491445,CoinMarketCap,CoinMarketCap,2013-12-24T16:50:12.000Z,1412168764284706822,DeFi Kingdoms,DefiKingdoms
7,1445334798977830912,Ruin,JustRuin_,2021-10-05T10:27:37.000Z,1412168764284706822,DeFi Kingdoms,DefiKingdoms
8,1446651877681106944,Albus 💙,dfk_albus,2021-10-09T01:41:10.000Z,1412168764284706822,DeFi Kingdoms,DefiKingdoms
9,1371438004020076547,MrZipper 💙,MrZipper7,2021-03-15T12:28:20.000Z,1412168764284706822,DeFi Kingdoms,DefiKingdoms


In [33]:
df_following_all.shape

(5308, 7)

In [34]:
df_following_all.head()

,id,name,username,created_at,following_id,following_name,following_username
0,189184866,"Fiskantes (⭐️,🩸)",Fiskantes,2010-09-10T16:08:14.000Z,1261684151780081664,Vulcan Forged,VulcanForged
1,1469146121482039301,Bifinity,bifinity,2021-12-10T03:28:03.000Z,1261684151780081664,Vulcan Forged,VulcanForged
2,1032885581049524226,Antler,AntlerGlobal,2018-08-24T07:01:27.000Z,1261684151780081664,Vulcan Forged,VulcanForged
3,1288572182444961793,Gala Games,GoGalaGames,2020-07-29T20:29:05.000Z,1261684151780081664,Vulcan Forged,VulcanForged
4,799169688,Magnus Grimeland,MagnusGrimeland,2012-09-02T22:41:30.000Z,1261684151780081664,Vulcan Forged,VulcanForged


In [35]:
data_folder = '../data/'
output_file = data_folder + 'who_are_projects_following.csv'
df_following_all.to_csv(output_file,index=False)